### helper notebook to show how to generate maps based off of preds.csv after running the pipeline

authored nov 14, 2025 by jelshawa

#### context
to run the pipeline and generate predictions starting from the latest noaa recorded time, execute this command
> !python main.py --config config.yaml

In [1]:
%%time
!python main.py --config config.yaml

2025-11-17 15:20:13,979 — INFO — 🔥 starting inference pipeline!
2025-11-17 15:20:13,980 — INFO — 📂 run dir: ./TEST
2025-11-17 15:20:13,980 — INFO — 📦 loading pretrained artifacts...
/Users/amritkrishnan/src/gaca-early-warning/.venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2025-11-17 15:20:25,979 — INFO — 🧠 loading trained model weights...
2025-11-17 15:20:25,984 — INFO — 🔧 using patched model_args: {'in_channels': 8, 'out_channels': 1, 'hidden_dim': 32, 'pred_horizons': 7, 'num_nodes': 14005}
2025-11-17 15:20:26,010 — INFO — loaded model GCNGRU with args: {'in_channels': 8, 'out_channels': 1, 'hidden_dim': 32, 'pred_horizons': 7, 'num_nodes': 14005}
2

#### imports + helper functions

In [2]:
"""Notebook helper imports for generating visualization maps from predictions CSV."""
import os
from argparse import Namespace
from datetime import timedelta

import numpy as np
import pandas as pd
import yaml

from util.plotting import plot_inference_maps, plot_prediction_timeseries

In [3]:
def load_config(config_path):
    """Load configuration from YAML file and convert to Namespace.

    Parameters
    ----------
    config_path : str
        Path to the YAML configuration file.

    Returns
    -------
    argparse.Namespace
        Configuration as a nested Namespace object with dotted-notation access.
    """
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)

    def to_namespace(obj):
        if isinstance(obj, dict):
            ns = Namespace()
            for k, v in obj.items():
                setattr(ns, k, to_namespace(v))
            return ns
        return obj

    return to_namespace(cfg)

#### main setup

In [4]:
args = load_config("./config.yaml")

In [5]:
nodes_csv_path = args.graph.nodes_csv_path

In [6]:
# load preds csv
pred_csv_path = f"{args.run_dir}/predictions.csv"
df_pred = pd.read_csv(pred_csv_path)

print("loaded preds csv:")
df_pred.head()

loaded preds csv:


,forecast_time,horizon_hours,lat,lon,predicted_temp
0,2025-11-17 14:00:00,1,42.000080,-80.84195,4.800504
1,2025-11-17 14:00:00,1,42.000153,-78.74701,-2.585595
2,2025-11-17 14:00:00,1,42.000496,-79.24889,-1.385997
3,2025-11-17 14:00:00,1,42.000782,-78.51062,-2.862100
4,2025-11-17 14:00:00,1,42.001110,-78.27423,-1.289757


In [7]:
# get num of horizons from csv
pred_offsets = sorted(df_pred["horizon_hours"].unique().tolist())
H = len(pred_offsets)

# get num of nodes
unique_nodes = df_pred[["lat", "lon"]].drop_duplicates()
N = len(unique_nodes)

print(f"horizons (H): {H}")
print(f"nodes (N): {N}")

horizons (H): 7
nodes (N): 14005


In [8]:
pred_tensor = np.zeros((1, H, N, 1), dtype=np.float32)

# sort df by horizon + (lat, lon)
df_sorted = df_pred.sort_values(["horizon_hours", "lat", "lon"]).reset_index(drop=True)

for h_idx, horizon in enumerate(pred_offsets):
    df_h = df_sorted[df_sorted["horizon_hours"] == horizon]
    pred_tensor[0, h_idx, :, 0] = df_h["predicted_temp"].values

In [9]:
# double check shape, should be (batch_size, # horizons, # nodes, output dim)
preds_unscaled_from_csv = pred_tensor
print(f"Shape: {preds_unscaled_from_csv.shape}")
# so for current setup should be (1, 7, 14005, 1)
# bc only doing one 'sequence', for 7 horizons, have 14005 nodes, and
# 1 pred for each node per horizon (temp)

Shape: (1, 7, 14005, 1)


In [10]:
# double check that num nodes match our nodes df
nodes_df = pd.read_csv(nodes_csv_path)
nodes_df = nodes_df.sort_values(["lat", "lon"]).reset_index(drop=True)
assert len(nodes_df) == N

In [11]:
# get latest timestamp that we used for generating the predictions
h_min = int(df_pred["horizon_hours"].min())  # min of the horizons
t_forecast_0 = pd.to_datetime(df_pred["forecast_time"]).min()  # min of the timestamps

latest_timestamp = t_forecast_0 - timedelta(
    hours=h_min
)  # latest ts = min of the timestamps - min horizon time

print("recovered latest_timestamp:", latest_timestamp)

recovered latest_timestamp: 2025-11-17 13:00:00


#### plotting

In [12]:
# create plot dir
plot_dir = f"{args.run_dir}/plots_from_csv"
os.makedirs(plot_dir, exist_ok=True)

# plot spatial maps
print("generating spatial inference maps from csv...")
plot_inference_maps(
    preds_unscaled=preds_unscaled_from_csv,
    nodes_df=nodes_df,
    pred_offsets=pred_offsets,
    out_dir=os.path.join(plot_dir, "maps"),
    latest_timestamp=latest_timestamp,
)

generating spatial inference maps from csv...
2025-11-17 15:21:27,204 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-17_14-00_plus1h.png
2025-11-17 15:21:27,341 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-17_19-00_plus6h.png
2025-11-17 15:21:27,478 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-18_01-00_plus12h.png
2025-11-17 15:21:27,631 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-18_07-00_plus18h.png
2025-11-17 15:21:27,768 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-18_13-00_plus24h.png
2025-11-17 15:21:27,905 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-19_01-00_plus36h.png
2025-11-17 15:21:28,042 — INFO — 📌 saved spatial map → ./TEST/plots_from_csv/maps/spatial_2025-11-19_13-00_plus48h.png


In [13]:
# plot timeseries
print("generating timeseries plots from csv...")
plot_prediction_timeseries(
    preds_unscaled=preds_unscaled_from_csv,
    pred_offsets=pred_offsets,
    out_dir=os.path.join(plot_dir, "timeseries"),
    latest_timestamp=latest_timestamp,
)

generating timeseries plots from csv...
2025-11-17 15:21:28,094 — INFO — 📌 saved mean-timeseries → ./TEST/plots_from_csv/timeseries/mean_timeseries_20251117_1300.png
2025-11-17 15:21:28,142 — INFO — 📌 saved node-timeseries → ./TEST/plots_from_csv/timeseries/node_9928_timeseries_20251117_1300.png
2025-11-17 15:21:28,187 — INFO — 📌 saved node-timeseries → ./TEST/plots_from_csv/timeseries/node_7976_timeseries_20251117_1300.png
2025-11-17 15:21:28,237 — INFO — 📌 saved node-timeseries → ./TEST/plots_from_csv/timeseries/node_5545_timeseries_20251117_1300.png
2025-11-17 15:21:28,283 — INFO — 📌 saved node-timeseries → ./TEST/plots_from_csv/timeseries/node_12972_timeseries_20251117_1300.png
2025-11-17 15:21:28,329 — INFO — 📌 saved node-timeseries → ./TEST/plots_from_csv/timeseries/node_6527_timeseries_20251117_1300.png
